In [1]:
#Fetches the data for the current Série A

import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# Make a request to the webpage URL and extract data from fourth table
url = 'https://www.chancedegol.com.br/br23.htm'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')[7]

# Loop through the rows of the table and extract the data from the cells
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Write the data to a CSV file
with open('dataBRA23raw.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerows(data)
    


In [2]:
df = pd.read_csv('dataBRA23raw.csv')
df

,Data,Mandante,Unnamed: 2,Visitante,Vitória doMandante,Empate,Vitória doVisitante
0,13/11/2022,Athletico PR,3x0,Botafogo,43.1 %,26.8 %,30.1 %
1,15/04/2023,Botafogo,2x1,São Paulo,36.9 %,27.9 %,35.2 %
2,15/04/2023,Palmeiras,2x1,Cuiabá,74.0 %,18.6 %,7.4 %
3,15/04/2023,Red Bull Bragantino,2x1,Bahia,56.7 %,23.7 %,19.6 %
4,15/04/2023,Atlético MG,1x2,Vasco,55.1 %,25.9 %,19.0 %
...,...,...,...,...,...,...,...
154,23/07/2023,Santos,2x2,Botafogo,16.7 %,25.8 %,57.5 %
155,23/07/2023,Vasco,0x2,Athletico PR,35.5 %,25.6 %,38.9 %
156,23/07/2023,Red Bull Bragantino,0x0,Internacional,48.2 %,27.4 %,24.3 %
157,23/07/2023,Cruzeiro,0x1,Goiás,58.6 %,26.6 %,14.7 %


In [3]:
# This output cleans and organizes the Data.

df.rename(columns={'Mandante':'Casa','Visitante':'Fora'}, inplace =True)
# Changes dtype to datetime in first dataframe
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
# Creates a new column with the for the competition name

df = df.assign(Comp='Série A')

# Separates the results of second dataframe into new columns
new = df["Unnamed: 2"].str.split("x", n = 1, expand = True)
df["GC"]= new[0] 
df["GF"]= new[1]

# Compares scores to display the results in the second dataframe
df.loc[df['GC'] == df['GF'], 'Res'] = 'E'
df.loc[df['GC'] > df['GF'], 'Res'] = 'VC' 
df.loc[df['GC'] < df['GF'], 'Res'] = 'VV'
# Create a new column to store the season information in the second data frame
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df['Temp'] = df['Data'].dt.year
# Organizes de columns of the sacond dataframe
new_order = ['Comp','Temp','Data','Casa','GC', 'GF','Fora','Res']
df= df.reindex(columns=new_order)


#Padronizes the names of the teams
df.replace({'América MG': 'América-MG',
            'Atlético MG': 'Atlético-MG',
            'Athletico PR': 'Athletico-PR',                   
            }, inplace=True)
df.dropna(axis=0, how='all', inplace=True)

# Sort values by date
df = df.sort_values(by='Data')

# Atributes a pontuation to each game
df['PC'] = df.apply(lambda x: 3 if x['Res'] == 'VC' else
                              1 if x['Res'] == 'E' else 0, axis=1)
df['PF'] = df.apply(lambda x: 3 if x['Res'] == 'VV' else
                              1 if x['Res'] == 'E' else 0, axis=1)

# Atributes a number of games for each season
df['J'] = df.groupby('Temp').cumcount() + 1
# Corrects any possible issue before saving as new file
df = df.dropna()
df = df.drop(index = 0)
df

C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_4680\35930777.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_4680\35930777.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_4680\35930777.py:20: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdep

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Série A,2023,2023-04-15,Botafogo,2,1,São Paulo,VC,3,0,1
2,Série A,2023,2023-04-15,Palmeiras,2,1,Cuiabá,VC,3,0,2
3,Série A,2023,2023-04-15,Red Bull Bragantino,2,1,Bahia,VC,3,0,3
4,Série A,2023,2023-04-15,Atlético-MG,1,2,Vasco,VV,0,3,4
5,Série A,2023,2023-04-15,Athletico-PR,2,0,Goiás,VC,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...
156,Série A,2023,2023-07-23,Red Bull Bragantino,0,0,Internacional,E,1,1,154
157,Série A,2023,2023-07-23,Cruzeiro,0,1,Goiás,VV,0,3,155
154,Série A,2023,2023-07-23,Santos,2,2,Botafogo,E,1,1,156
155,Série A,2023,2023-07-23,Vasco,0,2,Athletico-PR,VV,0,3,157


In [4]:
#Saves the data into a CSV format
df.to_csv('dataBRA23.csv', index=False)
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Série A,2023,2023-04-15,Botafogo,2,1,São Paulo,VC,3,0,1
2,Série A,2023,2023-04-15,Palmeiras,2,1,Cuiabá,VC,3,0,2
3,Série A,2023,2023-04-15,Red Bull Bragantino,2,1,Bahia,VC,3,0,3
4,Série A,2023,2023-04-15,Atlético-MG,1,2,Vasco,VV,0,3,4
5,Série A,2023,2023-04-15,Athletico-PR,2,0,Goiás,VC,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...
156,Série A,2023,2023-07-23,Red Bull Bragantino,0,0,Internacional,E,1,1,154
157,Série A,2023,2023-07-23,Cruzeiro,0,1,Goiás,VV,0,3,155
154,Série A,2023,2023-07-23,Santos,2,2,Botafogo,E,1,1,156
155,Série A,2023,2023-07-23,Vasco,0,2,Athletico-PR,VV,0,3,157
